# Example of Metric Learning in Embedded Space

In [1]:
%load_ext autoreload
%autoreload 2

# System imports
import os
import sys
import yaml
import logging
logging.basicConfig(level=logging.ERROR)  

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import torch
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning import Trainer
import torch.nn.functional as F
import frnn
from torch_scatter import scatter_add, scatter_mean, scatter_max

sys.path.append("../../../")

from LightningModules.SuperEmbedding.Models.undirected_embedding import UndirectedEmbedding
from LightningModules.SuperEmbedding.Models.gravmetric2 import GravMetric

device = "cuda" if torch.cuda.is_available() else "cpu"

## Roadmap

- Check that dataset has 1GeV cut
- Run baseline training of embedding: check eff/pur
- Build undirected GravMetric model
- Test run!
- Build and run undirected half-twin: check eff/pur
- Check definition of directed truth
- Build and run directed half-twin: check eff/pur
- If directed works, build and test directed GravMetric

In [2]:
# Load config file
with open("configs/vanilla_embedding.yaml", "r") as f:
    hparams = yaml.load(f)

<ipython-input-2-3b8c708bff14>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  hparams = yaml.load(f)


## The Dataset

In [3]:
model = UndirectedEmbedding(hparams)
model.setup(stage="fit")

100%|██████████| 120/120 [00:02<00:00, 42.44it/s]


In [4]:
model.setup(stage="fit")
trainset, valset, testset = model.trainset, model.valset, model.testset

In [5]:
model.trainset, model.valset, model.testset = trainset, valset, testset

## Baseline

In [3]:
model = UndirectedEmbedding(hparams)

In [4]:
logger = WandbLogger(
    project=hparams["project"], group="InitialTest", save_dir=hparams["artifacts"]
)
trainer = Trainer(
    gpus=1,
    max_epochs=hparams["max_epochs"],
    logger=logger,
)
trainer.fit(model)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
100%|██████████| 120/120 [00:01<00:00, 63.40it/s]
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: murnanedaniel. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.trainer.connectors.signal_connector:Set SLURM handle signals.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 3.2 M 
---------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.706    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performa

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## Train GravMetric

In [2]:
# Load config file
with open("configs/gravmetric.yaml", "r") as f:
    hparams = yaml.load(f)

<ipython-input-2-d64c37093f2f>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  hparams = yaml.load(f)


In [3]:
model  = GravMetric(hparams)

In [4]:
model.setup(stage="fit")
trainset, valset, testset = model.trainset, model.valset, model.testset

100%|██████████| 120/120 [00:03<00:00, 37.87it/s]


In [4]:
logger = WandbLogger(
    project=hparams["project"], group="InitialTest", save_dir=hparams["artifacts"]
)

trainer = Trainer(
    gpus=1,
    max_epochs=hparams["max_epochs"],
    logger=logger,
    num_sanity_val_steps=0,
)

trainer.fit(model)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
100%|██████████| 120/120 [00:01<00:00, 75.44it/s]
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: murnanedaniel. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.trainer.connectors.signal_connector:Set SLURM handle signals.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type       | Params
-----------------------------------------------
0 | feature_encoder | Sequential | 3.6 K 
1 | grav_convs      | ModuleList | 436 K 
2 | output_network  | Sequential | 266 K 
-----------------------------------------------
706 K     Trainable params
0         Non-trainable params
706 K     Total params
2.826     Total estimated model params size (MB)
/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the 

Training: 0it [00:00, ?it/s]

/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Validating: 0it [00:00, ?it/s]

/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Validating: 0it [00:00, ?it/s]

Error in callback <bound method AutoreloadMagics.post_execute_hook of <autoreload.AutoreloadMagics object at 0x155551271a60>> (for post_execute):


/global/homes/d/danieltm/.conda/envs/exatrkx-gpu/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


KeyboardInterrupt: 

In [24]:
from torch_geometric.nn import radius, radius_graph, knn, knn_graph

In [18]:
%%time
radius(trainset[0].x, trainset[0].x, r=0.1, max_num_neighbors=32).shape

CPU times: user 16.9 ms, sys: 0 ns, total: 16.9 ms
Wall time: 14.1 ms


torch.Size([2, 118863])

In [27]:
%%time
knn(trainset[0].x, trainset[0].x, k=2).shape

CPU times: user 3.08 ms, sys: 3.8 ms, total: 6.89 ms
Wall time: 6.06 ms


torch.Size([2, 13754])

In [19]:
%%time
radius_graph(trainset[0].x, r=0.1, max_num_neighbors=32).shape

CPU times: user 29.9 ms, sys: 0 ns, total: 29.9 ms
Wall time: 15.7 ms


torch.Size([2, 113959])

In [15]:
radius_graph(trainset[0].x, r=0.1, max_num_neighbors=32).shape

torch.Size([2, 113959])

In [26]:
knn_graph(trainset[0].x,k=2).shape

torch.Size([2, 13754])

In [59]:
query = torch.Tensor([[1, 0]])
database = torch.Tensor([[0, 0], [1, 0]])

In [60]:
query

tensor([[1., 0.]])

In [61]:
database

tensor([[0., 0.],
        [1., 0.]])

In [63]:
radius(database, query, r=0.1)

tensor([[0],
        [1]])